#Set up a classical optimization solver

Here we see the classical optimization solver used in VQE or QAOA quantum-classical hybrid algorithms.

In [21]:
!pip install blueqat

When we calculate the ansatz we just set up the minimizer.

In [22]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.pauli import qubo_bit as q
from blueqat.vqe import AnsatzBase, Vqe, get_scipy_minimizer

class QubitAnsatz(AnsatzBase):
    def __init__(self, hamiltonian):
        super().__init__(hamiltonian, 4)
        self.step = 1

    def get_circuit(self, params):
        a, b, c, d = params
        return Circuit().ry(a)[0].rz(b)[0].ry(c)[1].rz(d)[1]

h = -3*q(0)-3*q(1)-2*q(0)*q(1)
h = h.to_expr().simplify()
minimizer = get_scipy_minimizer(method="COBYLA",options={"tol":5.0e-4})
runner = Vqe(QubitAnsatz(h),minimizer=minimizer)
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy(result.circuit, runner.sampler))

# Hamiltonian to matrix
mat = h.to_matrix()

# Calculate by numpy
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

Result by VQE
-7.999999738356178
Result by numpy
-8.0


##Other solvers
If you want to use your own solver, it can be set up by making a function. This time we try to use bayesian optimization solver "hyperopt".

In [24]:
!pip install hyperopt

In [0]:
def hyperopt_minimizer(objective, n_params):
    from hyperopt import fmin, Trials, tpe, hp
    trials = Trials()
    best = fmin(objective, [hp.uniform(f'p{i}', 0., 2 * np.pi) for i in range(n_params)],
            algo=tpe.suggest, max_evals=100, trials=trials, verbose=1)
    return list(best.values())

In [26]:
runner = Vqe(QubitAnsatz(h),minimizer=hyperopt_minimizer)
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy(result.circuit, runner.sampler))

# Hamiltonian to matrix
mat = h.to_matrix()

# Calculate by numpy
print('Result by numpy')
print(np.linalg.eigh(mat)[0][0])

100%|██████████| 100/100 [00:01<00:00, 87.43it/s, best loss: -7.879139761619782]
Result by VQE
-7.879139761619782
Result by numpy
-8.0


We succeded to set up an optimizer.